# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [1]:
# Initial imports
import os
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
import nltk as nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from newsapi import NewsApiClient
from matplotlib import pyplot as plt

In [2]:
# download lexicon
nltk.download('vader_lexicon')

# init sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/stuartperry/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
# Read your api key environment variable
path = Path('/Users/stuartperry/Env_files/API_keys/news_api.env')
load_dotenv(path)
api_key = os.getenv('NEWS_API_KEY')
type(api_key) 

str

In [4]:
# Create a newsapi client
news_api = NewsApiClient(api_key=api_key)

In [5]:
# Fetch the Bitcoin news articles
btc_articles = news_api.get_everything(
    q='bitcoin, BTC, btc',
    language='en',
    sort_by='relevancy',
    page_size=100
)

In [6]:
# Fetch the Ethereum news articles
eth_articles = news_api.get_everything(
    q='ethereum, ETH, eth',
    language='en',
    sort_by='relevancy',
    page_size=100
)

In [7]:
# Create the Bitcoin sentiment scores DataFrame
btc_df = pd.DataFrame.from_dict(btc_articles["articles"])
btc_df.shape

(100, 8)

In [8]:
# Create the Ethereum sentiment scores DataFrame
eth_df = pd.DataFrame.from_dict(eth_articles["articles"])
eth_df.shape

(100, 8)

In [9]:
# Describe the Bitcoin Sentiment
# apply polarity scores along the text content 
btc_df['sentiment'] = btc_df['content'].apply(analyzer.polarity_scores)

# unpack dict in rows into df
btc_sentiment = pd.json_normalize(btc_df['sentiment'])

btc_sentiment.describe()

,neg,neu,pos,compound
count,100.000000,100.00000,100.000000,100.000000
mean,0.039920,0.89833,0.061750,0.081996
std,0.053537,0.07319,0.057697,0.364675
min,0.000000,0.61400,0.000000,-0.680800
25%,0.000000,0.86300,0.000000,-0.157050
50%,0.000000,0.91400,0.052500,0.137850
75%,0.067250,0.95000,0.083500,0.307000
max,0.217000,1.00000,0.285000,0.868200


In [10]:
# Describe the Ethereum Sentiment
# apply polarity scores function along the content column 
eth_df['sentiment'] = eth_df['content'].apply(analyzer.polarity_scores)

# unpack dict in rows into df
eth_sentiment = pd.json_normalize(eth_df['sentiment'])

eth_sentiment.describe()

,neg,neu,pos,compound
count,100.000000,100.000000,100.000000,100.000000
mean,0.032580,0.881250,0.086190,0.208627
std,0.046563,0.082281,0.078291,0.412814
min,0.000000,0.635000,0.000000,-0.784500
25%,0.000000,0.832000,0.000000,0.000000
50%,0.000000,0.887500,0.073000,0.249850
75%,0.058000,0.936000,0.133250,0.514625
max,0.231000,1.000000,0.332000,0.899400


### Questions:

Results from 2022-05-11, using the top 100 most relevant articles 

Q: Which coin had the highest mean positive score?

A: Ethereum has a mean positive score of 0.086 which is slightly higher than bitcoin's mean positive score of 0.062

Q: Which coin had the highest compound score?

A: Ethereum had the nhighest maximum compounds score 0.899, compared to bitcoin's 0.868

Q. Which coin had the highest positive score?

A: Ethereum also ends up on top with a maximum positive score of 0.332, when compared to bitcoin's 0.285

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [11]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [12]:
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()

# Create a list of stopwords
sw = set(stopwords.words('english'))

In [14]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    
    # only keep alphabetical characters
    regex = re.compile("[^a-zA-Z ]")
    no_punct = regex.sub('', text)
   
    # Create a tokenized list of the words
    tokens = word_tokenize(no_punct)

    # Lemmatize words into root words, apply lower and filter by stopwords
    tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens if word.lower() not in sw]

    return tokens

In [15]:
# apply tokenizer along content column, create a new tokens column for Bitcoin
btc_df['tokens'] = btc_df['content'].apply(tokenizer)
btc_df.head()

,source,author,title,description,url,urlToImage,publishedAt,content,sentiment,tokens
0,"{'id': 'wired', 'name': 'Wired'}",Arielle Pardes,Miami’s Bitcoin Conference Left a Trail of Har...,"For some women, inappropriate conduct from oth...",https://www.wired.com/story/bitcoin-2022-confe...,https://media.wired.com/photos/627a89e3e37e715...,2022-05-10T16:59:46Z,"Now, even though there are a number of women-f...","{'neg': 0.0, 'neu': 0.964, 'pos': 0.036, 'comp...","[even, though, number, womenfocused, crypto, s..."
1,"{'id': None, 'name': 'Bitcoin.com'}",None,"Bitcoin, Ethereum Technical Analysis: BTC Slip...","<ol><li>Bitcoin, Ethereum Technical Analysis: ...",https://news.bitcoin.com/bitcoin-ethereum-tech...,https://static.news.bitcoin.com/wp-content/upl...,2022-05-09T13:45:30Z,"BTC fell to its lowest level since last July, ...","{'neg': 0.066, 'neu': 0.859, 'pos': 0.076, 'co...","[btc, fell, lowest, level, since, last, july, ..."
2,"{'id': None, 'name': 'Investorplace.com'}",InvestorPlace,3 Reddit Stocks That Could Roar in Q2,The trick to sifting through Reddit stocks is ...,https://investorplace.com/2022/04/3-reddit-sto...,https://images.readwrite.com/wp-content/upload...,2022-04-19T11:01:00Z,<ul><li>Elon Musk will be able to focus on Tes...,"{'neg': 0.0, 'neu': 0.921, 'pos': 0.079, 'comp...","[ullielon, musk, able, focus, onteslatsla, pro..."
3,"{'id': 'business-insider', 'name': 'Business I...",prosen@insider.com (Phil Rosen),10 things before the opening bell,Today we're breaking down why top Wall Street ...,https://www.businessinsider.com/10-things-befo...,https://i.insider.com/6270066486fa90001905ff5d...,2022-05-03T10:00:00Z,Good morning. The combo of hawkish Fed policy ...,"{'neg': 0.118, 'neu': 0.713, 'pos': 0.17, 'com...","[good, morning, combo, hawkish, fed, policy, h..."
4,"{'id': None, 'name': 'Due.com'}",John Rampton,Protecting Retirement Savings from Volatile Cr...,Did you hear? You may be able to allocate some...,https://due.com/blog/protecting-retirement-sav...,https://assets.entrepreneur.com/providers/due/...,2022-05-09T14:00:04Z,Did you hear? You may be able to allocate some...,"{'neg': 0.0, 'neu': 0.955, 'pos': 0.045, 'comp...","[hear, may, able, allocate, k, retirement, sav..."


In [16]:
# apply tokenizer along content column, create a new tokens column for Ethereum
eth_df['tokens'] = eth_df['content'].apply(tokenizer)
eth_df.head()

,source,author,title,description,url,urlToImage,publishedAt,content,sentiment,tokens
0,"{'id': 'the-verge', 'name': 'The Verge'}",Emma Roth,US blames North Korean hacker group for $625 m...,The US Department of Treasury has attributed t...,https://www.theverge.com/2022/4/14/23025739/no...,https://cdn.vox-cdn.com/thumbor/n8TGCqcud7UHPI...,2022-04-14T21:43:57Z,The US Department of Treasury says Lazarus is ...,"{'neg': 0.231, 'neu': 0.681, 'pos': 0.088, 'co...","[u, department, treasury, say, lazarus, behind..."
1,"{'id': None, 'name': 'Bitcoin.com'}",None,"Bitcoin, Ethereum Technical Analysis: BTC Slip...","<ol><li>Bitcoin, Ethereum Technical Analysis: ...",https://news.bitcoin.com/bitcoin-ethereum-tech...,https://static.news.bitcoin.com/wp-content/upl...,2022-05-09T13:45:30Z,"BTC fell to its lowest level since last July, ...","{'neg': 0.066, 'neu': 0.859, 'pos': 0.076, 'co...","[btc, fell, lowest, level, since, last, july, ..."
2,"{'id': None, 'name': 'The Guardian'}",Alex Hern,Yuga Labs apologises after sale of virtual lan...,Demand for plots of land in multiplayer game O...,https://amp.theguardian.com/technology/2022/ma...,https://i.guim.co.uk/img/media/e6382b02449503a...,2022-05-02T15:54:35Z,A multi-billion dollar cryptocurrency company ...,"{'neg': 0.075, 'neu': 0.879, 'pos': 0.046, 'co...","[multibillion, dollar, cryptocurrency, company..."
3,"{'id': None, 'name': 'CNET'}",Daniel Van Boom,How Bored Ape Yacht Club Broke Ethereum - CNET,"Transactions stalled for hours, and people pai...",https://www.cnet.com/personal-finance/crypto/h...,https://www.cnet.com/a/img/resize/ec8610207cf7...,2022-05-04T02:32:00Z,When Bored Ape Yacht Club creators Yuga Labs a...,"{'neg': 0.055, 'neu': 0.945, 'pos': 0.0, 'comp...","[bored, ape, yacht, club, creator, yuga, lab, ..."
4,"{'id': 'the-next-web', 'name': 'The Next Web'}",Ivan Mehta,How the BAYC metaverse mint raised Ethereum ga...,If you ever wanted to buy an NFT based on Ethe...,https://thenextweb.com/news/bayc-metaverse-oth...,https://img-cdn.tnwcdn.com/image/hardfork?filt...,2022-05-02T12:17:07Z,If you ever wanted to buy an NFT based on Ethe...,"{'neg': 0.036, 'neu': 0.964, 'pos': 0.0, 'comp...","[ever, wanted, buy, nft, based, ethereum, woul..."


---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [17]:
from collections import Counter
from nltk import ngrams

In [16]:
# Generate the Bitcoin N-grams where N=2


In [17]:
# Generate the Ethereum N-grams where N=2
# YOUR CODE HERE!

In [18]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [19]:
# Use token_count to get the top 10 words for Bitcoin
# YOUR CODE HERE!

In [20]:
# Use token_count to get the top 10 words for Ethereum
# YOUR CODE HERE!

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [21]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [22]:
# Generate the Bitcoin word cloud
# YOUR CODE HERE!

In [23]:
# Generate the Ethereum word cloud
# YOUR CODE HERE!

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [24]:
import spacy
from spacy import displacy

In [25]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [26]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [27]:
# Concatenate all of the Bitcoin text together
# YOUR CODE HERE!

In [28]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [29]:
# Render the visualization
# YOUR CODE HERE!

In [30]:
# List all Entities
# YOUR CODE HERE!

---

### Ethereum NER

In [31]:
# Concatenate all of the Ethereum text together
# YOUR CODE HERE!

In [32]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [33]:
# Render the visualization
# YOUR CODE HERE!

In [34]:
# List all Entities
# YOUR CODE HERE!

---